In [2]:
import re

import requests
import pdfplumber
import pandas as pd
from collections import namedtuple

In [25]:
pdf1 = 'apreports.pdf'

In [26]:
with pdfplumber.open(pdf1) as pdf:
    page = pdf.pages[16]
    text = page.extract_text()

In [27]:
print(text)

AccountsPayableReportPack
Invoice by Vendor List
Date: 08/23/2022 Invoice by Vendor List Page: 1
Jensen, Martin & Anderson, P.C.
Inv Due Inv Disc Disc Net
Voucher # Invoice # Date Date Amount Amount Date Amount 1099 Ref # Description Codes
200 United Parcel Service
200.01MLJ 3243387 032422 062422 75.00 P 75.00 1 Overnight document delivery * 1
3243387 032522 062522 75.00 P 75.00 3 Postage * 1
3243387 072422 072422 75.00 P 75.00 2 Overnight document delivery * 1
Vendor Totals 225.00 0.00 225.00
202 Software Technology, LLC
312205 081822 082322 490.00 P 490.00 1 Software Maintenance Agreement Tabs3 * 1
312206 081822 082322 239.00 P 239.00 2 Software Maintenance Agreement Tabs3 General Ledger * 1
312207 081822 082322 239.00 P 239.00 3 Software Maintenance Agreement Tabs3 Accounts Payable * 1
312208 081822 082322 239.00 P 239.00 4 Software Maintenance Agreement Tabs3 Trust Accounting * 1
312209 081822 082322 800.00 P 800.00 5 Software Maintenance Agreement PracticeMaster (9 Users) * 1
3122

In [30]:
vendors= re.compile(r'^\d{3} [A-Z].*')

In [29]:
for i in text.split('\n'):
    if vendors.match(i):
        print(i)

200 United Parcel Service
202 Software Technology, LLC
203 Clerk of the County Court
204 Clerk of the District Court
225 Jackson/Wylinda
240 NELCO
250 D & B Real Estate Management Company
300 Thomson Reuters Payment Center
325 Professional Messenger Services
400 Sprint Local & Long Distance
700 Clean All Janitorial Services


In [54]:
for i in text.split('\n'):
    if vendors.match(i):
        vend_num, *vend_name = i.split()
        vend_name = ' '.join(vend_name)
    print(vend_num)
    print(vend_name)


700
Clean All Janitorial Services
700
Clean All Janitorial Services
700
Clean All Janitorial Services
700
Clean All Janitorial Services
700
Clean All Janitorial Services
700
Clean All Janitorial Services
200
United Parcel Service
200
United Parcel Service
200
United Parcel Service
200
United Parcel Service
200
United Parcel Service
202
Software Technology, LLC
202
Software Technology, LLC
202
Software Technology, LLC
202
Software Technology, LLC
202
Software Technology, LLC
202
Software Technology, LLC
202
Software Technology, LLC
202
Software Technology, LLC
202
Software Technology, LLC
203
Clerk of the County Court
203
Clerk of the County Court
203
Clerk of the County Court
204
Clerk of the District Court
204
Clerk of the District Court
204
Clerk of the District Court
204
Clerk of the District Court
204
Clerk of the District Court
225
Jackson/Wylinda
225
Jackson/Wylinda
225
Jackson/Wylinda
225
Jackson/Wylinda
240
NELCO
240
NELCO
240
NELCO
250
D & B Real Estate Management Company
250


In [108]:
format = re.compile(r'(\d{6}) (\d{6}) ([\d,]+\.\d{2}) [\sP]*([\d,]+\.\d{2}) [YN ]*\d (.*?) [*\s\d]')

In [109]:
names = namedtuple('names', 'vend_num vend_name inv_dt due_dt inv_amt net_amt description')

In [122]:
items = []
for i in text.split('\n'):
    if vendors.search(i):
        vend_num, *vend_name = i.split()
        vend_name = ' '.join(vend_name)    
    
    i = format.search(i)    
    if i:        
        inv_dt = i.group(1)
        due_dt = i.group(2)
        inv_amt = i.group(3)
        net_amt = i.group(4)
        desc = i.group(5)
        items.append(names(vend_num, vend_name, inv_dt, due_dt, inv_amt, net_amt, desc))

In [123]:
df = pd.DataFrame(items)
df

,vend_num,vend_name,inv_dt,due_dt,inv_amt,net_amt,description
0,200,United Parcel Service,032422,062422,75.00,75.00,Overnight document delivery
1,200,United Parcel Service,032522,062522,75.00,75.00,Postage
2,200,United Parcel Service,072422,072422,75.00,75.00,Overnight document delivery
3,202,"Software Technology, LLC",081822,082322,490.00,490.00,Software Maintenance Agreement Tabs3
4,202,"Software Technology, LLC",081822,082322,239.00,239.00,Software Maintenance Agreement Tabs3 General L...
5,202,"Software Technology, LLC",081822,082322,239.00,239.00,Software Maintenance Agreement Tabs3 Accounts ...
6,202,"Software Technology, LLC",081822,082322,239.00,239.00,Software Maintenance Agreement Tabs3 Trust Acc...
7,202,"Software Technology, LLC",081822,082322,800.00,800.00,Software Maintenance Agreement PracticeMaster ...
8,202,"Software Technology, LLC",081822,082322,105.00,105.00,Software Maintenance Agreement Tabs3 Taskbill
9,202,"Software Technology, LLC",082322,082822,205.00,205.00,Purchase Tabs3 Device Interface Program


In [118]:
df['inv_dt'] = pd.to_datetime(df['inv_dt'])
df['due_dt'] = pd.to_datetime(df['due_dt'])
df

,vend_num,vend_name,inv_dt,due_dt,inv_amt,net_amt,description
0,200,United Parcel Service,2022-03-24,2022-06-24,75.00,75.00,Overnight document delivery
1,200,United Parcel Service,2022-03-25,2022-06-25,75.00,75.00,Postage
2,200,United Parcel Service,2022-07-24,2022-07-24,75.00,75.00,Overnight document delivery
3,202,"Software Technology, LLC",2022-08-18,2022-08-23,490.00,490.00,Software Maintenance Agreement Tabs3
4,202,"Software Technology, LLC",2022-08-18,2022-08-23,239.00,239.00,Software Maintenance Agreement Tabs3 General L...
5,202,"Software Technology, LLC",2022-08-18,2022-08-23,239.00,239.00,Software Maintenance Agreement Tabs3 Accounts ...
6,202,"Software Technology, LLC",2022-08-18,2022-08-23,239.00,239.00,Software Maintenance Agreement Tabs3 Trust Acc...
7,202,"Software Technology, LLC",2022-08-18,2022-08-23,800.00,800.00,Software Maintenance Agreement PracticeMaster ...
8,202,"Software Technology, LLC",2022-08-18,2022-08-23,105.00,105.00,Software Maintenance Agreement Tabs3 Taskbill
9,202,"Software Technology, LLC",2022-08-23,2022-08-28,205.00,205.00,Purchase Tabs3 Device Interface Program


In [121]:
df.to_csv('invoice.csv')